In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [62]:
gm = pd.read_csv('all_metrc_gm_0720.csv',index_col = 0)

In [63]:
gm['company_identifier'].unique()

array(['99HT', 'AGA', 'ALT', 'BS', 'BMC', 'BBF', 'BUD', 'BRC', 'BYN',
       'CHO', 'CG', 'CPA', 'CPC', 'CSC', 'CSCC', 'CCC', 'CC', 'CED',
       'DWF', 'DCO', 'DL', 'DGG', 'DG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT',
       'EPC', 'EH', 'FI', 'FW', 'GTC', 'GT', 'GLNR', 'GRG', 'GTR', 'GHC',
       'GFEE', 'GFWF', 'HB', 'HS', 'HPCC', 'HC', 'IDC', 'ID', 'KAR', 'KC',
       'LEG', 'LBC', 'MSS', 'ML', 'MW', 'MPW', 'MD', 'ND', 'NECC', 'PGM',
       'PRP', 'PRL', 'QR', 'MC', 'RA', 'RGD', 'SGF', 'SFV', 'SV', 'SLCC',
       'SO', 'ST', 'DW', 'FD', 'TGL', 'TJR', 'TTS', 'TT', 'T4L', 'TL',
       'UHHC', 'VS', 'VOY'], dtype=object)

In [64]:
gm_high_coverage = gm[gm['coverage'] >= 0.7]

In [65]:
gm_high_coverage_3m_trim = gm_high_coverage[(gm_high_coverage['gm_past_quarter'] > 0.2) & (gm_high_coverage['gm_past_quarter'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]

gm_high_coverage_6m_trim = gm_high_coverage[(gm_high_coverage['gm_past_2quarters'] > 0.2) & (gm_high_coverage['gm_past_2quarters'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]

gm_high_coverage_9m_trim = gm_high_coverage[(gm_high_coverage['gm_past_3quarters'] > 0.2) & (gm_high_coverage['gm_past_3quarters'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]



In [66]:
state_count = gm_high_coverage_3m_trim[['state','id','date']].groupby(['state','date']).count().unstack().T.reset_index()
state_count



state,level_0,date,CA,CO,MA,MI,OR
0,id,2020-03,15.0,8.0,NaN,NaN,NaN
1,id,2020-04,16.0,10.0,NaN,1.0,1.0
2,id,2020-05,18.0,9.0,NaN,2.0,1.0
3,id,2020-06,20.0,8.0,NaN,2.0,NaN
4,id,2020-07,22.0,9.0,NaN,3.0,1.0
5,id,2020-08,24.0,10.0,1.0,3.0,1.0
6,id,2020-09,26.0,10.0,1.0,2.0,1.0
7,id,2020-10,26.0,9.0,1.0,2.0,1.0
8,id,2020-11,27.0,10.0,1.0,1.0,1.0
9,id,2020-12,27.0,10.0,1.0,2.0,NaN


In [70]:
gm_high_coverage_3m_trim.sort_values(by ='gm_past_3quarters_after_tax',ascending = False )

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,gm_past_quarter,gm_past_2quarters,gm_past_3quarters,gm_past_quarter_after_tax,gm_past_2quarters_after_tax,gm_past_3quarters_after_tax,gm$_past_quarter_after_tax,gm$_past_2quarters_after_tax,gm$_past_3quarters_after_tax,company,location,id,company_identifier,state
132,2020-09,29284.09,9121.787308,20162.302692,0.688507,604.0,0.0,604,0.755000,33676.7035,11516.256476,22160.447024,0.658035,0.711121,0.798632,0.838895,0.682861,0.778933,0.823135,31186.034893,45716.153428,39132.478576,BMC,402-00930,"('BMC', '402-00930')",BMC,CO
173,2022-01,82835.61,19979.341961,62856.268039,0.758807,1633.0,0.0,1633,0.749770,95260.9515,25223.919226,70037.032274,0.735212,0.763759,0.785559,0.815460,0.740649,0.764581,0.797407,61683.670693,69703.661417,71867.289572,BBF,MR281525,"('BBF', 'MR281525')",BBF,MA
175,2022-03,88019.31,17778.082914,70241.227086,0.798021,1824.0,0.0,1824,0.714734,101222.2065,22444.829678,78777.376822,0.778262,0.782757,0.775198,0.796428,0.761505,0.753206,0.776513,72371.534473,67143.530716,71596.918890,BBF,MR281525,"('BBF', 'MR281525')",BBF,MA
2358,2020-09,496394.21,136383.951034,360010.258966,0.725251,15011.0,43.0,14968,0.997939,570853.3415,172184.738180,398668.603320,0.698373,0.720377,0.743423,0.790463,0.693023,0.718323,0.769965,434670.352323,461041.203735,467245.040959,SFV,C10-0000041-LIC,"('SFV', 'C10-0000041-LIC')",SFV,CA
176,2022-04,90868.54,24338.023532,66530.516468,0.732162,2194.0,0.0,2194,0.710492,104498.8210,30726.754709,73772.066291,0.705961,0.773875,0.768817,0.781664,0.751754,0.746201,0.760305,73616.545812,67650.108253,71007.956215,BBF,MR281525,"('BBF', 'MR281525')",BBF,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,2021-12,32715.38,12190.853894,20524.526106,0.627366,1064.0,131.0,933,0.921212,37622.6870,15390.953041,22231.733959,0.590913,0.676775,NaN,NaN,0.645155,NaN,NaN,16598.526271,NaN,NaN,VOY,C10-0000802-LIC,"('VOY', 'C10-0000802-LIC')",VOY,CA
3056,2022-01,24339.01,11142.403847,13196.606153,0.542200,833.0,45.0,788,0.939121,27989.8615,14067.284857,13922.576643,0.497415,0.625583,NaN,NaN,0.588956,NaN,NaN,16931.354154,NaN,NaN,VOY,C10-0000802-LIC,"('VOY', 'C10-0000802-LIC')",VOY,CA
3057,2022-02,24450.82,10455.332937,13995.487063,0.572393,803.0,44.0,759,0.961677,28118.4430,13199.857834,14918.585166,0.530562,0.580653,0.604680,NaN,0.539630,0.566007,NaN,17024.298590,14628.196498,NaN,VOY,C10-0000802-LIC,"('VOY', 'C10-0000802-LIC')",VOY,CA
3058,2022-03,27920.68,11694.072300,16226.607700,0.581168,952.0,73.0,879,0.966497,32108.7820,14763.766279,17345.015721,0.540195,0.565254,0.621014,NaN,0.522724,0.583940,NaN,15395.392510,15996.959391,NaN,VOY,C10-0000802-LIC,"('VOY', 'C10-0000802-LIC')",VOY,CA


## CA

In [8]:
### 3m

In [9]:
gm_high_coverage_3m_trim_ca = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'CA']

In [10]:
margin_distr_3m_ca = gm_high_coverage_3m_trim_ca.groupby(['date'])['gm_past_quarter_after_tax'].describe().reset_index()
margin_distr_3m_ca['date'] = pd.to_datetime(margin_distr_3m_ca['date'])



In [11]:
#margin_distr_3m_ca[['25%','50%','75%']].mean().round(2)

In [12]:
margin_distr_3m_ca[margin_distr_3m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.46
50%    0.52
75%    0.60
dtype: float64

In [13]:
### 6m

In [14]:
gm_high_coverage_6m_trim_ca = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'CA']
margin_distr_6m_ca = gm_high_coverage_6m_trim_ca.groupby(['date'])['gm_past_2quarters_after_tax'].describe().reset_index()
margin_distr_6m_ca['date'] = pd.to_datetime(margin_distr_6m_ca['date'])

#margin_distr_6m_ca[['25%','50%','75%']].mean().round(2)



In [15]:
margin_distr_6m_ca[margin_distr_6m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.46
50%    0.51
75%    0.60
dtype: float64

In [16]:
### 9m

In [17]:
gm_high_coverage_9m_trim_ca = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'CA']
margin_distr_9m_ca = gm_high_coverage_9m_trim_ca.groupby(['date'])['gm_past_3quarters_after_tax'].describe().reset_index()
margin_distr_9m_ca['date'] = pd.to_datetime(margin_distr_9m_ca['date'])

#margin_distr_9m_ca[['25%','50%','75%']].mean().round(2)



In [18]:
margin_distr_9m_ca[margin_distr_9m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.46
50%    0.51
75%    0.60
dtype: float64

## CO

In [ ]:
### 3m

In [19]:
gm_high_coverage_3m_trim_co = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'CO']

In [20]:
margin_distr_3m_co = gm_high_coverage_3m_trim_co.groupby(['date'])['gm_past_quarter'].describe().reset_index()
margin_distr_3m_co['date'] = pd.to_datetime(margin_distr_3m_co['date'])



In [21]:
margin_distr_3m_co[['25%','50%','75%']].mean().round(2)

25%    0.48
50%    0.57
75%    0.62
dtype: float64

In [22]:
margin_distr_3m_co[margin_distr_3m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.47
50%    0.56
75%    0.63
dtype: float64

In [23]:
### 6m

In [24]:
gm_high_coverage_6m_trim_co = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'CO']
margin_distr_6m_co = gm_high_coverage_6m_trim_co.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
margin_distr_6m_co['date'] = pd.to_datetime(margin_distr_6m_co['date'])

margin_distr_6m_co[['25%','50%','75%']].mean().round(2)



25%    0.47
50%    0.57
75%    0.62
dtype: float64

In [25]:
margin_distr_6m_co[margin_distr_6m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.47
50%    0.57
75%    0.63
dtype: float64

In [26]:
### 9m

In [27]:
gm_high_coverage_9m_trim_co = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'CO']
margin_distr_9m_co = gm_high_coverage_9m_trim_co.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
margin_distr_9m_co['date'] = pd.to_datetime(margin_distr_9m_co['date'])

margin_distr_9m_co[['25%','50%','75%']].mean().round(2)



25%    0.47
50%    0.57
75%    0.62
dtype: float64

In [28]:
margin_distr_9m_co[margin_distr_9m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.48
50%    0.57
75%    0.63
dtype: float64

## MI

In [ ]:
### 3m

In [29]:
gm_high_coverage_3m_trim_mi = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'MI']
margin_distr_3m_mi = gm_high_coverage_3m_trim_mi.groupby(['date'])['gm_past_quarter'].describe().reset_index()
margin_distr_3m_mi['date'] = pd.to_datetime(margin_distr_3m_mi['date'])

margin_distr_3m_mi[['25%','50%','75%']].mean().round(2)



25%    0.45
50%    0.50
75%    0.56
dtype: float64

In [30]:
margin_distr_3m_mi[margin_distr_3m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.42
50%    0.48
75%    0.54
dtype: float64

In [31]:
### 6m

In [32]:
gm_high_coverage_6m_trim_mi = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'MI']
margin_distr_6m_mi = gm_high_coverage_6m_trim_mi.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
margin_distr_6m_mi['date'] = pd.to_datetime(margin_distr_6m_mi['date'])

margin_distr_6m_mi[['25%','50%','75%']].mean().round(2)



25%    0.52
50%    0.56
75%    0.59
dtype: float64

In [33]:
margin_distr_6m_mi[margin_distr_6m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.48
50%    0.52
75%    0.57
dtype: float64

In [34]:
### 9m

In [35]:
gm_high_coverage_9m_trim_mi = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'MI']
margin_distr_9m_mi = gm_high_coverage_9m_trim_mi.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
margin_distr_9m_mi['date'] = pd.to_datetime(margin_distr_9m_mi['date'])

margin_distr_9m_mi[['25%','50%','75%']].mean().round(2)



25%    0.54
50%    0.57
75%    0.59
dtype: float64

In [36]:
margin_distr_9m_mi[margin_distr_9m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.51
50%    0.55
75%    0.57
dtype: float64

## MA

In [ ]:
### 3m

In [37]:
gm_high_coverage_3m_trim_ma = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'MA']
margin_distr_3m_ma = gm_high_coverage_3m_trim_ma.groupby(['date'])['gm_past_quarter'].describe().reset_index()
margin_distr_3m_ma['date'] = pd.to_datetime(margin_distr_3m_ma['date'])

margin_distr_3m_ma[['25%','50%','75%']].mean().round(2)



25%    0.53
50%    0.54
75%    0.55
dtype: float64

In [38]:
margin_distr_3m_ma[margin_distr_3m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.55
75%    0.56
dtype: float64

In [39]:
### 6m

In [40]:
gm_high_coverage_6m_trim_ma = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'MA']
margin_distr_6m_ma = gm_high_coverage_6m_trim_ma.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
margin_distr_6m_ma['date'] = pd.to_datetime(margin_distr_6m_ma['date'])

margin_distr_6m_ma[['25%','50%','75%']].mean().round(2)



25%    0.54
50%    0.54
75%    0.56
dtype: float64

In [41]:
margin_distr_6m_ma[margin_distr_6m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.55
75%    0.56
dtype: float64

In [42]:
### 9m

In [43]:
gm_high_coverage_9m_trim_ma = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'MA']
margin_distr_9m_ma = gm_high_coverage_9m_trim_ma.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
margin_distr_9m_ma['date'] = pd.to_datetime(margin_distr_9m_ma['date'])

margin_distr_9m_ma[['25%','50%','75%']].mean().round(2)



25%    0.54
50%    0.54
75%    0.56
dtype: float64

In [44]:
margin_distr_9m_ma[margin_distr_9m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.54
75%    0.56
dtype: float64

In [45]:
df_clients = [
'DL',
'DW',
'EMA',
'EMM',
'EMT',
'EMF',
'ST',
'GRG',
'EL',
'VS',
'TT',
'MD',
'DWF',
'GHC',
'SV',
'99HT',
'NECC',
'MPW',
'LBC'
]

data_client = gm[gm['company_identifier'].isin(df_clients)]

In [47]:
gm_high_coverage = data_client[data_client['coverage'] >= 0.7]

gm_high_coverage_3m_trim = gm_high_coverage[(gm_high_coverage['gm_past_quarter'] > 0.2) & (gm_high_coverage['gm_past_quarter'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]

gm_high_coverage_6m_trim = gm_high_coverage[(gm_high_coverage['gm_past_2quarters'] > 0.2) & (gm_high_coverage['gm_past_2quarters'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]

gm_high_coverage_9m_trim = gm_high_coverage[(gm_high_coverage['gm_past_3quarters'] > 0.2) & (gm_high_coverage['gm_past_3quarters'] < 0.8) & (gm_high_coverage['date'] < '2022-07-01')]



In [48]:
## CA

### 3m

gm_high_coverage_3m_trim_ca = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'CA']

margin_distr_3m_ca = gm_high_coverage_3m_trim_ca.groupby(['date'])['gm_past_quarter_after_tax'].describe().reset_index()
margin_distr_3m_ca['date'] = pd.to_datetime(margin_distr_3m_ca['date'])



#margin_distr_3m_ca[['25%','50%','75%']].mean().round(2)

margin_distr_3m_ca[margin_distr_3m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.49
50%    0.56
75%    0.60
dtype: float64

In [49]:
### 6m

gm_high_coverage_6m_trim_ca = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'CA']
margin_distr_6m_ca = gm_high_coverage_6m_trim_ca.groupby(['date'])['gm_past_2quarters_after_tax'].describe().reset_index()
margin_distr_6m_ca['date'] = pd.to_datetime(margin_distr_6m_ca['date'])

#margin_distr_6m_ca[['25%','50%','75%']].mean().round(2)



margin_distr_6m_ca[margin_distr_6m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.49
50%    0.56
75%    0.60
dtype: float64

In [50]:
### 9m

gm_high_coverage_9m_trim_ca = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'CA']
margin_distr_9m_ca = gm_high_coverage_9m_trim_ca.groupby(['date'])['gm_past_3quarters_after_tax'].describe().reset_index()
margin_distr_9m_ca['date'] = pd.to_datetime(margin_distr_9m_ca['date'])

#margin_distr_9m_ca[['25%','50%','75%']].mean().round(2)



margin_distr_9m_ca[margin_distr_9m_ca['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.50
50%    0.56
75%    0.60
dtype: float64

In [51]:
## CO

### 3m

gm_high_coverage_3m_trim_co = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'CO']

margin_distr_3m_co = gm_high_coverage_3m_trim_co.groupby(['date'])['gm_past_quarter'].describe().reset_index()
margin_distr_3m_co['date'] = pd.to_datetime(margin_distr_3m_co['date'])
margin_distr_3m_co[['25%','50%','75%']].mean().round(2)

margin_distr_3m_co[margin_distr_3m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.44
50%    0.46
75%    0.51
dtype: float64

In [52]:


### 6m

gm_high_coverage_6m_trim_co = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'CO']
margin_distr_6m_co = gm_high_coverage_6m_trim_co.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
margin_distr_6m_co['date'] = pd.to_datetime(margin_distr_6m_co['date'])

margin_distr_6m_co[['25%','50%','75%']].mean().round(2)



margin_distr_6m_co[margin_distr_6m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.42
50%    0.45
75%    0.51
dtype: float64

In [53]:
### 9m

gm_high_coverage_9m_trim_co = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'CO']
margin_distr_9m_co = gm_high_coverage_9m_trim_co.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
margin_distr_9m_co['date'] = pd.to_datetime(margin_distr_9m_co['date'])

margin_distr_9m_co[['25%','50%','75%']].mean().round(2)



margin_distr_9m_co[margin_distr_9m_co['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.42
50%    0.44
75%    0.50
dtype: float64

In [59]:
# ## MI

# ### 3m

# gm_high_coverage_3m_trim_mi = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'MI']
# margin_distr_3m_mi = gm_high_coverage_3m_trim_mi.groupby(['date'])['gm_past_quarter'].describe().reset_index()
# margin_distr_3m_mi['date'] = pd.to_datetime(margin_distr_3m_mi['date'])

# margin_distr_3m_mi[['25%','50%','75%']].mean().round(2)



# margin_distr_3m_mi[margin_distr_3m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

In [60]:
# ### 6m

# gm_high_coverage_6m_trim_mi = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'MI']
# margin_distr_6m_mi = gm_high_coverage_6m_trim_mi.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
# margin_distr_6m_mi['date'] = pd.to_datetime(margin_distr_6m_mi['date'])

# margin_distr_6m_mi[['25%','50%','75%']].mean().round(2)



# margin_distr_6m_mi[margin_distr_6m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

In [ ]:
# ### 9m

# gm_high_coverage_9m_trim_mi = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'MI']
# margin_distr_9m_mi = gm_high_coverage_9m_trim_mi.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
# margin_distr_9m_mi['date'] = pd.to_datetime(margin_distr_9m_mi['date'])

# margin_distr_9m_mi[['25%','50%','75%']].mean().round(2)



# margin_distr_9m_mi[margin_distr_9m_mi['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

In [56]:
## MA

### 3m

gm_high_coverage_3m_trim_ma = gm_high_coverage_3m_trim[gm_high_coverage_3m_trim['state'] == 'MA']
margin_distr_3m_ma = gm_high_coverage_3m_trim_ma.groupby(['date'])['gm_past_quarter'].describe().reset_index()
margin_distr_3m_ma['date'] = pd.to_datetime(margin_distr_3m_ma['date'])

margin_distr_3m_ma[['25%','50%','75%']].mean().round(2)



margin_distr_3m_ma[margin_distr_3m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.55
75%    0.55
dtype: float64

In [57]:
### 6m

gm_high_coverage_6m_trim_ma = gm_high_coverage_6m_trim[gm_high_coverage_6m_trim['state'] == 'MA']
margin_distr_6m_ma = gm_high_coverage_6m_trim_ma.groupby(['date'])['gm_past_2quarters'].describe().reset_index()
margin_distr_6m_ma['date'] = pd.to_datetime(margin_distr_6m_ma['date'])

margin_distr_6m_ma[['25%','50%','75%']].mean().round(2)



margin_distr_6m_ma[margin_distr_6m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.54
75%    0.55
dtype: float64

In [58]:
### 9m

gm_high_coverage_9m_trim_ma = gm_high_coverage_9m_trim[gm_high_coverage_9m_trim['state'] == 'MA']
margin_distr_9m_ma = gm_high_coverage_9m_trim_ma.groupby(['date'])['gm_past_3quarters'].describe().reset_index()
margin_distr_9m_ma['date'] = pd.to_datetime(margin_distr_9m_ma['date'])

margin_distr_9m_ma[['25%','50%','75%']].mean().round(2)



margin_distr_9m_ma[margin_distr_9m_ma['date'] >= '2021-01-01'][['25%','50%','75%']].mean().round(2)

25%    0.54
50%    0.54
75%    0.54
dtype: float64